In [ ]:
%pip install langchain openai transformers --quiet
%pip install -U langchain-openai --quiet
%pip install torch torchvision torchaudio --quiet
%pip install -U langchain-community --quiet
%pip install ipywidgets --quiet
%pip install pandas --quiet
%pip install scikit-learn --quiet
%pip install matplotlib Pillow --quiet

In [ ]:
import os
import io
import pandas as pd
from PIL import Image as PILImage, ImageEnhance, ImageFilter
from transformers import CLIPProcessor, CLIPModel
import torch
from langchain.agents import initialize_agent
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.tools import BaseTool
import chromadb
import base64
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

In [ ]:
# Set OpenAI API key
os.environ["OPENAI_API_KEY"] = "sk-proj-uRoYTQ11Qi4yndbUFWSOT3BlbkFJ5dcSHgZPnJRjSpXUwRU7"

# Initialising the model and database
gpt4o = ChatOpenAI(temperature=0, model_name="gpt-4o")

text_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialising ChromaDB
chroma_client = chromadb.PersistentClient(path="./image_vdb")

embedding_function = SentenceTransformerEmbeddingFunction()
image_summaries_collection = chroma_client.get_or_create_collection(name="image_summaries", embedding_function=embedding_function)

In [ ]:
# Image Preprocessing Functions
def preprocess_image(image_path):
    image = PILImage.open(image_path).convert('RGB')

    # Resizing images
    image = image.resize((224, 224))
 
    # Enhanced contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(2)

    # Denoising
    image = image.filter(ImageFilter.MedianFilter(size=3))

    return image

def generate_image_summary(image_path):
    try:
        img = PILImage.open(image_path)
        
        if os.path.getsize(image_path) > 20 * 1024 * 1024:  # 20MB
            raise ValueError("Image file is too large (>20MB)")
        
        if img.format.lower() not in ['png', 'jpeg', 'jpg', 'gif', 'webp']:
            raise ValueError(f"Unsupported image format: {img.format}")
        
        preprocessed_image = preprocess_image(image_path)
        
        buffered = io.BytesIO()
        preprocessed_image.save(buffered, format="JPEG")
        image_base64 = base64.b64encode(buffered.getvalue()).decode('utf-8')
        
        response = gpt4o.invoke([
            HumanMessage(content=[
                {"type": "text", "text": """Analyze this image of a mechanical component and provide a brief summary. Focus on:
                1. Key visual features: shape, size, surface texture, and any visible details.
                2. Potential manufacturing method: based on the component's appearance, suggest the most likely manufacturing process.
                3. Any visible text or markings.
                Format your response as follows:
                Visual Features: [List key visual characteristics]
                Likely Manufacturing Method: [State the most probable manufacturing process and briefly explain why]
                Text/Markings: [Mention any visible text or markings, or state if none are visible]"""},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}}
            ])
        ])
                            
        return response.content
    except Exception as e:
        raise ValueError(f"Error processing image {image_path}: {str(e)}")
    finally:
        if 'img' in locals():
            img.close()

In [ ]:
def clear_vdb():
    global image_collection
    try:
        chroma_client.delete_collection(name="image_collection")
        print("Existing collection deleted.")
    except ValueError:
        print("No existing collection found. Proceeding to create a new one.")
    
    image_collection = chroma_client.get_or_create_collection(
        name="image_collection", 
        embedding_function=embedding_function
    )
    print("New collection created.")

def generate_text_embedding(text):
    return text_model.encode(text).tolist()

def add_to_vectorstore(image_path, text_info):
    try:
    
        image_summary = generate_image_summary(image_path)
        summary_embedding = generate_text_embedding(image_summary)
        
        text_embedding = generate_text_embedding(text_info)
        
        combined_embedding = [(s + t) / 2 for s, t in zip(summary_embedding, text_embedding)]
        
        image_collection.add(
            documents=[image_summary],
            embeddings=[combined_embedding],
            metadatas=[{"image_path": image_path, "text_info": text_info}],
            ids=[image_path]
        )
        return True  
    except Exception as e:
        print(f"Error processing image {image_path}: {str(e)}")
        return False


def add_images_to_vectorstore(directory_paths, max_images_per_folder=2000):
    for directory_path in directory_paths:
        images_processed = 0 
        skipped_images = 0
        for filename in sorted(os.listdir(directory_path)):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.webp')):
                image_path = os.path.join(directory_path, filename)
                text_info = os.path.basename(directory_path)
                
                if add_to_vectorstore(image_path, text_info):
                    images_processed += 1
                    print(f"Processed {images_processed} images from {directory_path}")
                else:
                    skipped_images += 1
                    print(f"Skipped image: {image_path}")
                
                if images_processed >= max_images_per_folder:
                    print(f"Reached maximum number of images for {directory_path}")
                    break
        
        print(f"Finished processing {directory_path}")
        print(f"Total processed: {images_processed}, Skipped: {skipped_images}")

local_image_dirs = [
    r'C:\Users\wanghanyu\Desktop\5_Classes\Fabricated',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Forged',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Other',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Sheet Metal',
    r'C:\Users\wanghanyu\Desktop\5_Classes\Turned'
]

clear_vdb()

add_images_to_vectorstore(local_image_dirs, max_images_per_folder=10)

In [ ]:
def query_db(image_path, text, results=3, top_k=5):
    # Generate a summary of the query images
    query_summary = generate_image_summary(image_path)
    summary_embedding = generate_text_embedding(query_summary)
    
    # Generate embedding for query text
    text_embedding = generate_text_embedding(text)
    
    # Combining two embedded
    query_embedding = [(s + t) / 2 for s, t in zip(summary_embedding, text_embedding)]
    
    query_results = image_collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    
    filtered_results = filter_results(query_results, image_path, top_k=results)
    
    visualize_results(image_path, filtered_results['metadatas'])
    
    return filtered_results

def filter_results(query_results, image_path, top_k):
    distances_and_uris_and_ids = list(zip(
        query_results['distances'][0], 
        [m['image_path'] for m in query_results['metadatas'][0]], 
        query_results['ids'][0]
    ))

    filtered_results = [
        (dist, uri, id) for dist, uri, id in distances_and_uris_and_ids
        if os.path.normpath(uri) != os.path.normpath(image_path) and dist != 0
    ]

    sorted_results = sorted(filtered_results, key=lambda x: x[0])

    unique_results = []
    seen_distances = set()
    for dist, uri, id in sorted_results:
        if dist not in seen_distances:
            unique_results.append((dist, uri, id))
            seen_distances.add(dist)

    final_results = unique_results[:top_k]

    return {
        'ids': [id for _, _, id in final_results],
        'uris': [uri for _, uri, _ in final_results],
        'distances': [dist for dist, _, _ in final_results],
        'metadatas': [{'image_path': uri} for _, uri, _ in final_results]
    }

def visualize_results(query_image_path, result_metadatas):
    fig, axs = plt.subplots(2, 2, figsize=(12, 12))
    fig.suptitle('Query Image and Top 3 Results', fontsize=16)

    query_img = PILImage.open(query_image_path).convert('RGB')
    axs[0, 0].imshow(query_img)
    axs[0, 0].set_title('Query Image')
    axs[0, 0].axis('off')

    for i, metadata in enumerate(result_metadatas[:3]):
        img = PILImage.open(metadata['image_path']).convert('RGB')
        row = (i + 1) // 2
        col = (i + 1) % 2
        axs[row, col].imshow(img)
        axs[row, col].set_title(f'Result {i+1}')
        axs[row, col].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
def format_prompt_inputs(data, user_query, text_info):
    inputs = {}
    inputs['user_query'] = user_query
    inputs['text_info'] = text_info

    image_path_1 = data['uris'][0]
    with open(image_path_1, 'rb') as image_file:
        image_data_1 = image_file.read()
    inputs['image_data_1'] = base64.b64encode(image_data_1).decode('utf-8')

    retrieved_texts = [os.path.basename(os.path.dirname(uri)) for uri in data['uris']]
    inputs['retrieved_info_texts'] = ', '.join(retrieved_texts)

    for i, uri in enumerate(data['uris'][:3]): 
        with open(uri, 'rb') as image_file:
            inputs[f'retrieved_info_images_{i}'] = base64.b64encode(image_file.read()).decode('utf-8')

    for i in range(3):
        if f'retrieved_info_images_{i}' not in inputs:
            inputs[f'retrieved_info_images_{i}'] = ''

    inputs['retrieved_info'] = {
        'uris': data['uris'],
        'distances': data['distances'],
        'texts': retrieved_texts,
    }

    return inputs

parser = StrOutputParser()

image_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a mechanical engineer specializing in manufacturing processes. Your task is to analyze images of mechanical components, identify their primary manufacturing method, and briefly describe key features of the component.

You will be provided with the main image to analyze, along with up to three similar components retrieved from a database. Your goal is to use the additional images for comparison and to enhance your analysis, providing a more comprehensive and accurate description."""),

    ("user", [
        {"type": "text", "text": """Analyze the main image of the mechanical component and identify its primary manufacturing method and key features.

You have access to up to 3 similar components retrieved from the database, the retrieved images are displayed below.

Consider the similarities and differences between the main image and the retrieved images to provide a more accurate and comprehensive description.

Respond with a concise sentence stating the primary manufacturing method used to produce this component and key features."""},
        {"type": "image_url", "image_url": "data:image/jpeg;base64,{image_data_1}"},
        {"type": "image_url", "image_url": "data:image/jpeg;base64,{retrieved_info_images_0}"},
        {"type": "image_url", "image_url": "data:image/jpeg;base64,{retrieved_info_images_1}"},
        {"type": "image_url", "image_url": "data:image/jpeg;base64,{retrieved_info_images_2}"},
    ]),
])

vision_chain = image_prompt | gpt4o | parser

def generate_description_with_rag(image_path):
    try:
        text_info = os.path.basename(os.path.dirname(image_path))
        user_query = "Analyze the mechanical component image with reference to similar components."
        results = query_db(image_path, text_info, results=3)
        print(f"Query results for {image_path}: {results}")

        if not results['ids']:
            print(f"Warning: No results retrieved for {image_path}")
            return "Unable to generate description due to lack of similar components."

        prompt_input = format_prompt_inputs(results, user_query, text_info)

        response = vision_chain.invoke(prompt_input)
        return response
    except Exception as e:
        print(f"Error in generate_description_with_rag for {image_path}: {str(e)}")
        return f"Error generating description: {str(e)}"

class DescribeImageWithDetailsTool(BaseTool):
    name = "Describe Image With Details Tool"
    description = 'Use this tool to describe an images of mechanical parts with additional details.'
    
    def _run(self, image_path: str):
        return self.run(image_path)
    
    def run(self, image_path: str):
        description = generate_description_with_rag(image_path)
        return description

tools = [DescribeImageWithDetailsTool()]
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=gpt4o,
    max_iterations=5,
    early_stopping_method='generate'
)

In [ ]:
def process_directories(directory_paths):
    results = []
    for directory_path in directory_paths:
        missing_files = []
        files_processed = 0
        for filename in os.listdir(directory_path):
            if filename.lower().endswith('.jpg'):
                files_processed += 1
                if files_processed > 1:
                    break
                image_path = os.path.join(directory_path, filename)
                print(f"Checking if the file exists: {image_path}")
                if os.path.exists(image_path):
                    try:
                        print(f"Calling agent to process the file: {image_path}")
                        description = generate_description_with_rag(image_path)
                        result = {"image_name": filename, "description": description}
                        results.append(result)
                    except Exception as e:
                        results.append({"image_name": filename, "description": f"Error processing file: {str(e)}"})
                else:
                    print(f"File not found: {image_path}")
                    missing_files.append(filename)
                    results.append({"image_name": filename, "description": "File not found"})
        if missing_files:
            print(f"The following files were not found in {directory_path}: {', '.join(missing_files)}")
    
    output_file_path = os.path.join(os.getcwd(), "RAG_LLMLLM.csv")
    df = pd.DataFrame(results)
    df.to_csv(output_file_path, index=False)
    print(f"CSV file saved at: {output_file_path}")
    return results
 
# Process the catalogue and generate a description
results = process_directories(local_image_dirs)

# Show results
for result in results:
    print(f"{result['image_name']}: {result['description']}")